In [1]:
# Add official website of pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import torch.nn.functional as F  # useful stateless functions

import numpy as np

USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [9]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class ResNetBatch(nn.Module):
    
    def __init__(self, in_channel, num_classes):
        super(ResNetBatch, self).__init__()
        
        self.conv_1 = nn.Conv2d(in_channel, 64, (7, 7), stride = 2, padding = 3)
        nn.init.kaiming_normal_(self.conv_1.weight)
        
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(3, stride = 2, padding = 1)
        
        self.residual_1 = nn.Conv2d(64, 64, kernel_size = 1)
        
        self.bn2_1 = nn.BatchNorm2d(64)
        self.bn2_2 = nn.BatchNorm2d(64)
        
        self.conv_2 = nn.Conv2d(64, 64, (3, 3), padding = 1)
        nn.init.kaiming_normal_(self.conv_2.weight)
        
        self.residual_2 = nn.Conv2d(64, 128, kernel_size = 1)
        
        self.conv_3_1 = nn.Conv2d(64, 128, (3, 3), padding = 1)
        nn.init.kaiming_normal_(self.conv_3_1.weight)
        
        self.conv_3_2 = nn.Conv2d(128, 128, (3, 3), padding = 1)
        nn.init.kaiming_normal_(self.conv_3_2.weight)
        
        self.bn3_1 = nn.BatchNorm2d(128)
        self.bn3_2 = nn.BatchNorm2d(128)
        
        self.residual_3 = nn.Conv2d(128, 256, kernel_size = 1)
        
        self.conv_4_1 = nn.Conv2d(128, 256, (3, 3), padding = 1)
        nn.init.kaiming_normal_(self.conv_4_1.weight)
        
        self.conv_4_2 = nn.Conv2d(256, 256, (3, 3), padding = 1)
        nn.init.kaiming_normal_(self.conv_4_2.weight)
        
        self.bn4_1 = nn.BatchNorm2d(256)
        self.bn4_2 = nn.BatchNorm2d(256)
        
        self.residual_4 = nn.Conv2d(256, 512, kernel_size = 1, stride = 2)
        
        self.conv_5_1 = nn.Conv2d(256, 512, (3, 3), stride = 2, padding = 1)
        nn.init.kaiming_normal_(self.conv_5_1.weight)
        
        self.conv_5_2 = nn.Conv2d(512, 512, (3, 3), padding = 1)
        nn.init.kaiming_normal_(self.conv_5_2.weight)
        
        self.bn5_1 = nn.BatchNorm2d(512)
        self.bn5_2 = nn.BatchNorm2d(512)
        
        self.avgpool = nn.AvgPool2d(3 + 1)
        self.fs = nn.Linear(18432, num_classes)
        
    def forward(self, x): 
        x = self.relu(self.bn1(self.conv_1(x)))
        x = self.maxpool(x)
        r1 = self.residual_1(x)
        
        x = self.relu(self.bn2_1(self.conv_2(x)))
        x = self.relu(self.bn2_2(self.conv_2(x)))
        r2 = x + r1
        
        x = self.relu(self.bn3_1(self.conv_3_1(r2)))
        x = self.relu(self.bn3_2(self.conv_3_2(x)))
        
        r3 = x + self.residual_2(r2)
        
        x = self.relu(self.bn4_1(self.conv_4_1(x)))
        x = self.relu(self.bn4_2(self.conv_4_2(x)))
        r4 = x + self.residual_3(r3)
        
        x = self.relu(self.bn5_1(self.conv_5_1(x)))
        x = self.relu(self.bn5_2(self.conv_5_2(x)))
        r5 = x + self.residual_4(r4)
        
        x = self.avgpool(r5)
        x = flatten(x)
        x = self.fs(x)
        
        return x
        

In [3]:
def check_accuracy_part34(loader, model):
    
    print('Checking accuracy on validation set')
    
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [4]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [5]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms, models 

from torch.utils.data import random_split

batch_size = 16
dataroot = "C:/Users/HinanawiTS/Documents/GitHub/ECE228-project/processed"
dataset = ImageFolder(root = dataroot, transform = transforms.Compose([transforms.ToTensor()]))
train_ratio = 0.7
trainlen = int(len(dataset)*train_ratio)
vallen = len(dataset)-trainlen
train, val = random_split(dataset, [trainlen, vallen])
loader_train = DataLoader(train, batch_size = batch_size, num_workers = 0, pin_memory = True)
loader_val = DataLoader(val, batch_size = batch_size, num_workers = 0, pin_memory = True)

In [ ]:
model = ResNetBatch(3, 90)
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
train_part34(model, optimizer, epochs = 10)

Epoch 0, Iteration 0, loss = 4.5370
Checking accuracy on validation set
Got 10 / 971 correct (1.03)

Epoch 0, Iteration 100, loss = 3.5040
Checking accuracy on validation set
Got 103 / 971 correct (10.61)

Epoch 1, Iteration 0, loss = 2.5702
Checking accuracy on validation set
Got 152 / 971 correct (15.65)

Epoch 1, Iteration 100, loss = 1.8223
Checking accuracy on validation set
Got 156 / 971 correct (16.07)

Epoch 2, Iteration 0, loss = 1.6527
Checking accuracy on validation set
Got 190 / 971 correct (19.57)

Epoch 2, Iteration 100, loss = 0.8483
Checking accuracy on validation set
Got 184 / 971 correct (18.95)

Epoch 3, Iteration 0, loss = 0.8410
Checking accuracy on validation set
Got 191 / 971 correct (19.67)

Epoch 3, Iteration 100, loss = 0.3791
Checking accuracy on validation set
Got 185 / 971 correct (19.05)

Epoch 4, Iteration 0, loss = 0.3748
Checking accuracy on validation set
Got 185 / 971 correct (19.05)

Epoch 4, Iteration 100, loss = 0.2176
Checking accuracy on validati

In [ ]:
import cv2
from os import listdir
import os

from PIL import Image

def csg(img, dim):
    width, height = img.shape[1], img.shape[0]


    crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
    mid_x, mid_y = int(width/2), int(height/2)
    cw2, ch2 = int(crop_width/2), int(crop_height/2) 
    crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
    return crop_img

i = 0
t = "animals/"
for an in listdir(t):
    th = t + an + "/"
    os.mkdir("processed/" + an)
    for fs in listdir(th): 
    
        i = i + 1
    
        print(th + fs)
    
        gna = cv2.imread(th + fs, cv2.IMREAD_UNCHANGED)
        if (gna.shape[0] > 200) and (gna.shape[1] > 200): 

            img = cv2.imread(th + fs, cv2.IMREAD_UNCHANGED)
            width = 200

            height = 200
            dim = (width, height)
  
            # resize image
            if gna.shape[0] < gna.shape[1]: 
                resiz = csg(img, (gna.shape[0], gna.shape[0]))
            else: 
            
                resiz = csg(img, (gna.shape[1], gna.shape[1]))
    
            resiz = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            print("processed/" + an + "/" + str(i) + ".jpg")
            cv2.imwrite("processed/" + an + "/" + str(i) + ".jpg", resiz)
        
